In [1]:
#Basado en: https://www.eduonix.com/dashboard/complete-guide-to-machine-learning-using-python
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import sys
import numpy as np

# conda install -c conda-forge rake_nltk
# conda install -c conda-forge scikit-surprise

movies = pd.read_csv('movies_metadata.csv', low_memory=False)

# print(movies["vote_average"].value_counts())

m_count = movies['vote_average'].quantile(0.85)
movies = movies[movies['vote_average'] > m_count]
movies = movies[movies['genres'].notnull()]
print(movies["genres"])

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
5        [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
15       [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
27       [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
28       [{'id': 14, 'name': 'Fantasy'}, {'id': 878, 'n...
31       [{'id': 878, 'name': 'Science Fiction'}, {'id'...
35                           [{'id': 18, 'name': 'Drama'}]
46       [{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...
49       [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
52       [{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...
57       [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
76                     [{'id': 99, 'name': 'Documentary'}]
79       [{'id': 10751, 'name': 'Family'}, {'id': 18, '...
94                          [{'id': 35, 'name': 'Comedy'}]
95                           [{'id': 18, 'name': 'Drama'}]
103      [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
108      [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam.

In [4]:
import ast

movies["genre_tags"] = ""

for index, row in movies.iterrows():
    genreTags = ast.literal_eval(row["genres"])

    genreTags = [tag['name'] for tag in genreTags]

    movies.loc[index,"genre_tags"] = " ".join(genreTags)
    
reviews = movies[["original_title","genre_tags"]]
titleInd = {i: movies.loc[i]["original_title"] for i in movies.index}

reviews = reviews.set_index('original_title')

In [5]:
## Memory-model (Euc Distance)
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import sys

indices = pd.Series(movies.index, index=movies['original_title']).drop_duplicates()
indiceList = reviews.index.tolist() # Indexes are not consistent range from 0 to N. 

count = TfidfVectorizer()
count_matrix = count.fit_transform(reviews['genre_tags'])

cosine_sim = euclidean_distances(count_matrix, count_matrix)

idx = indices["Heat"]
recommendations = list(enumerate((cosine_sim[idx])))

recommendations = sorted(recommendations, key=lambda x: x[1])[0:11]

for ind, weight in recommendations:
    print(indiceList[ind],weight)

Twelve Monkeys 0.0
Gattaca 0.359185698634441
パプリカ 0.4606763667307808
The Butterfly Effect 0.5358065235598273
Predestination 0.5358065235598273
What Happened to Monday 0.5358065235598273
Say Nothing 0.5431234579073518
Inception 0.5630907637488047
Blade Runner 0.5801832463821154
Сталкер 0.5801832463821154
The Day the Earth Stood Still 0.5801832463821154
